## Ussing GPT API to fix the scrapped recipe
As the recipes are missing the description, rating and quantity and dose, we decided one using GPT to fill out the gap

### Setup
For that, we need :
- the API key for GPT
- The Firebase crediential json

In [ ]:
import openai
import json
import firebase_admin
from firebase_admin import credentials, firestore

# Load your API key from an environment variable or secret management service
openai.api_key = 'The key'

# Load the data from the provided JSON file
file_path = './formatted_recipes_final.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Initialize Firestore DB
cred = credentials.Certificate('./firebase_credentials.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

Now let's load the recipe that need to be fixed

In [1]:
def fetch_recipes():
    recipes_ref = db.collection("recipes")
    return [doc for doc in recipes_ref.stream()]

recipe_docs = fetch_recipes()
print(len(recipe_docs))

1953


There is 1953 recipes to fix ! <br>
GPT take around 30sec by request and cost around 0.02 cent! <br>
Hence we need to filter our recipe to get only relevant one! <br>

In [3]:
# Most popular recipe among our dataset
recipe_names = [
    "Breakfast Waffles", "Chocolate Chip Pancakes I", "Coffee Cake", "Basted Egg", "Boiled Sweet Potato",
    "Black Bean Soup", "Burrito Mix", "Chocolate Chip Cookies III", "Basic Mayonnaise", "Tomato Basil Soup",
    "Veggie Stir-Fry", "Tuna Salad Sandwich", "BLT Sandwich", "Egg Fried Rice", "Garlic Butter Shrimp",
    "Avocado Toast", "BBQ Pulled Pork Sandwiches", "Beef Burgers with Chipotle Mayo", "Banana Pecan Bread",
    "Chicken Noodle Soup", "Chicken Fajitas", "Lemon Pepper Salmon", "Chicken and Waffles", "Cheese Omelette",
    "Shrimp Tacos", "French Toast", "Caprese Salad", "Veggie Sushi Rolls", "Chicken Shawarma", "Tuna Salad Sandwich",
    "Garlic Butter Shrimp", "BBQ Chicken Sandwich", "Chocolate Milkshake", "Veggie Omelette",
    "Cheese and Crackers", "Tomato Basil Soup", "Beef Tacos", "Chicken Parmesan",
    "Chicken Salad Sandwich", "French Fries", "Caesar Salad", "BBQ Wings", "Chicken Stir-Fry", "Spinach Salad",
    "Breakfast Casserole", "Tuna Salad Sandwich", "Egg Salad Sandwich", "Greek Salad", "Chicken Salad",
    "Chicken Quesadilla", "Chicken Wraps", "Chicken Salad Wrap", "BBQ Pulled Pork Sandwich", "Chicken Caesar Wrap",
    "Shrimp Salad", "Chicken Salad Sandwich", "Chicken Salad Wrap", "BBQ Chicken Sandwich", "Shrimp Tacos",
    "Avocado Toast", "Cheese and Crackers", "Chicken Salad", "Tuna Salad", "Veggie Salad", "Spinach Salad",
    "Caesar Salad", "Chicken Caesar Salad", "Greek Salad", "Tomato Soup", "Chicken Salad", "Tuna Salad",
    "Shrimp Salad", "Avocado Salad", "Greek Salad", "Caesar Salad", "Spinach Salad", "Chicken Caesar Salad",
    "Chicken Salad Wrap", "Tuna Salad Wrap", "BBQ Chicken Sandwich", "Shrimp Tacos", "Avocado Toast",
    "Cheese and Crackers", "Shrimp Salad", "Tuna Salad Sandwich",
    "Greek Salad", "Caesar Salad",     "Crunchy Baked Chicken",
    "Beef Fajitas",
    "Black Bean Soup",
    "Breakfast Waffles",
    "Coffee Cake",
    "Basted Egg",
    "Corn and Shrimp Chowder",
    "Chipotle Chili",
    "Banana Caramel Dessert",
    "Chocolate Milkshake",
    "A Nice Cup of Tea",
    "Classic Lasagna"
    "Banana Milkshake",
    "Banoffee Pie",
    "Cheese Omelette",
    "Breakfast Burrito",
    "1-2-3-4 Cake",
    "Baked Brunch Omelet",
    "Banana Chocolate Bread Pudding",
    "Chicken Noodle Soup",
    "Biscuits III",
    "Chocolate Chip Cookies III",
    "Carrot and Coriander Soup",
    "Coconut Crusted Baked Cod",
    "Buttermilk Fried Chicken with Pan Gravy",
    "Beer-can Chicken",
    "Chilaquiles",
    "Baked Sweet Potatoes",
    "Crispy Gingerbread Cookies",
    "Banana Pudding",
    "Braised Collard Greens",
    "Blueberry Muffin",
    "Biscuits I",
    "Baked Ziti",
    "Chicken Fajitas",
    "Baked Oatmeal",
    "Cauliflower Rice",
    "Cinnamon Rolls",
    "Cheese Egg Toast",
    "Chicken Bog I",
    "Apple Pie I",
    "Baked Penne",
    "Chicken and Broccoli Casserole",
    "Churros I",
    "Chocolate Chip Muffins",
    "Blueberry Pancakes",
    "Banana Cake",
    "Bread Pudding",
    "Basic Mayonnaise",
    "Cheesy Potato Soup",
    "Chicken Tetrazzini",
    "Coleslaw II",
    "Chicken Curry",
    "Crispy Fish Sandwich",
    "Chicken Marinade",
    "Chicken Alfredo",
    "Crispy Breaded Burgers",
    "Banana Chip Muffins",
    "Corn Chowder I",
    "Cheese-Stuffed Chicken Breasts",
    "Chocolate Bananas Foster",
    "Chocolate Chocolate Chip Muffins",
    "Cheese Spaetzle",
    "Baked Potatoes",
    "Baked Chicken Breasts",
    "Chocolate Chip Cookies I",
    "Boiled Sweet Potato",
    "Boiled Plantain with Vegetable Sauce",
    "Cheese Crisps",
    "Chicken and Rice Casserole",
    "Chicken Soup with Rice and Potatoes",
    "Crispy Buffalo Shrimp",
    "Crispy Chicken Burgers",
    "Chicken Riggies",
    "Chicken Alfredo Pizza",
    "Chicken Bacon Broccoli Alfredo Spaghetti",
    "Classic Pot Roast",
    "Chili Powder I",
    "Chicken Fried Rice",
    "Chicken Soup with Cream and Lemon",
    "Chicken Cacciatore",
    "Cheese and Bacon Rolls",
    "Banana Split II",
    "Crispy Roast Potatoes",
    "Cornbread and Chicken Casserole",
    "Chicken Quesadilla"
]

# Initialize an empty list to store the filtered recipes
doc_filter = []

# Filter the recipes
for doc in recipe_docs:
    recipe = doc.to_dict()
    if "title" in recipe and recipe["title"] in recipe_names:
        doc_filter.append(recipe)

# Now doc_filter contains the 100 filtered recipes
print((len(recipe_docs)))
print(len(doc_filter))


1953
182


In [4]:
# Let's look at them !
for recipe in doc_filter:
    if "title" in recipe:
        print(recipe)

{'imageUrl': '', 'tags': ['Chicken recipes', 'Cookbook', 'Ingredients', 'Recipes', 'Easy recipes', 'Recipes', 'Recipes with metric units', 'Chicken breast recipes', 'Butter recipes', 'Bread crumb recipes', 'Recipes without servings', 'Recipes without cooking time'], 'description': '', 'recipeId': '0CDtRnB2PvoQ9XkHs25Z', 'ingredients': [{'ingredient': {'vegetarian': False, 'quantity': 0, 'measure': 'NONE', 'vegan': False, 'id': '6Tjm54WJt8oJ5hVQYyfb', 'name': 'Butter'}}, {'ingredient': {'vegetarian': False, 'quantity': 0, 'measure': 'NONE', 'vegan': False, 'id': 'XLw3JNvO8FwPMERYd85v', 'name': 'Cheese'}}, {'ingredient': {'vegetarian': False, 'quantity': 0, 'measure': 'NONE', 'vegan': False, 'id': 'pcFWI1v6p643lHu4ZVoW', 'name': 'Chicken'}}, {'ingredient': {'vegetarian': False, 'quantity': 0, 'measure': 'NONE', 'vegan': False, 'id': 'FTmy39NJNezwDPKlRHYg', 'name': 'Parmesan cheese'}}, {'ingredient': {'vegetarian': False, 'quantity': 0, 'measure': 'NONE', 'vegan': False, 'id': 'qH6X8yxlPL

Now, we are going to ask gpt to fix them, we will give him an example, and prompt that explain the problem

In [ ]:

# Measure units of our App data model
measure_units = [
    "TEASPOON", "TABLESPOON", "CUP", "G", "KG", "L", "ML", "NONE", "EMPTY", "PIECES"
]

def correct_with_openai(recipe):
    example_recipe = {
        "recipeId": "example_id",
        "ingredients": [
            {"quantity": 500.0, "measure": "G", "ingredient": {"vegetarian": False, "vegan": False, "id": "", "name": "Ground beef"}},
            {"quantity": 500.0, "measure": "G", "ingredient": {"vegetarian": False, "vegan": False, "id": "", "name": "Tomato sauce"}},
            {"quantity": 1.0, "measure": "CUP", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Ricotta cheese"}},
            {"quantity": 2.0, "measure": "CUP", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Mozzarella cheese, shredded"}},
            {"quantity": 0.5, "measure": "CUP", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Parmesan cheese, grated"}},
            {"quantity": 1.0, "measure": "NONE", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Egg"}},
            {"quantity": 1.0, "measure": "TEASPOON", "ingredient": {"vegetarian": True, "vegan": True, "id": "", "name": "Salt"}},
            {"quantity": 1.0, "measure": "TEASPOON", "ingredient": {"vegetarian": True, "vegan": True, "id": "", "name": "Black pepper"}},
            {"quantity": 12.0, "measure": "NONE", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Lasagna noodles"}}
        ],
        "tags": ["Italian", "Pasta", "Dinner"],
        "description": "A classic Italian lasagna with layers of ground beef, tomato sauce, and three cheeses.",
        "imageUrl": "example_url",
        "rating": 4.9,
        "title": "Classic Lasagna",
        "steps": [
            {"description": "Preheat oven to 375°F (190°C).", "stepNumber": 1, "title": "Preheat Oven"},
            {"description": "In a large skillet, cook ground beef over medium heat until browned. Add tomato sauce and simmer for 10 minutes.", "stepNumber": 2, "title": "Cook Beef and Sauce"},
            {"description": "In a mixing bowl, combine ricotta cheese, egg, salt, and pepper.", "stepNumber": 3, "title": "Prepare Ricotta Mixture"},
            {"description": "Cook lasagna noodles according to package instructions. Drain and set aside.", "stepNumber": 4, "title": "Cook Noodles"},
            {"description": "In a 9x13 inch baking dish, spread a layer of meat sauce. Add a layer of noodles, followed by a layer of the ricotta mixture, mozzarella cheese, and Parmesan cheese. Repeat layers, ending with mozzarella and Parmesan on top.", "stepNumber": 5, "title": "Assemble Lasagna"},
            {"description": "Cover with aluminum foil and bake in preheated oven for 25 minutes. Remove foil and bake for an additional 25 minutes until cheese is bubbly and slightly golden.", "stepNumber": 6, "title": "Bake Lasagna"},
            {"description": "Let lasagna cool for 10 minutes before serving.", "stepNumber": 7, "title": "Cool and Serve"}
        ],
        "userid": "12345"
    }

    prompt = f"""
    Correct and fill out the following recipe JSON to ensure it contains all required fields, especially a description, step title and the quantity measure unit which should be one of the following {', '.join(measure_units)}. 
    If the measure is NONE, replace this value by a coherent one in function of the ingredient name.
    If the quantity is 0, replace this value by a coherent one in function of the ingredient name.
    If the rating is 0, based on the popularity of the recipe generate a rating.

    Example of the expected format:
    {json.dumps(example_recipe, indent=2)}

    Recipe to correct:
    {json.dumps(recipe, indent=2)}

    Output only the corrected recipe JSON. Ensure the output is valid JSON.
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that corrects recipe formats."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=3000
        )
        corrected_recipe_str = response['choices'][0]['message']['content'].strip()

        # Ensure the response is a valid JSON
        start_index = corrected_recipe_str.find('{')
        end_index = corrected_recipe_str.rfind('}') + 1
        corrected_recipe_str = corrected_recipe_str[start_index:end_index]

        corrected_recipe = json.loads(corrected_recipe_str)
        return corrected_recipe, True
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return recipe, False
    except Exception as e:
        print(f"Error correcting recipe with OpenAI: {e}")
        return recipe, False

def update_recipe_in_firestore(doc_id, corrected_recipe):
    db.collection("recipesFinal").document(doc_id).set(corrected_recipe, merge=True)

def ree():
    
    i = 0
    for doc in doc_filter:
        doc_id = doc['recipeId']
        i =+ 1
        print(f"------------- {i} -------------------")

        # Further correction with OpenAI if necessary
        corrected_recipe, ai_corrected = correct_with_openai(doc)
        if ai_corrected:
            print(f"Old recipe {doc}")
            print(f"Updated recipe {corrected_recipe}")
            update_recipe_in_firestore(doc_id, corrected_recipe)
            print(f"Recipe {doc_id} corrected and updated in Firestore.")
        else:
            print(f"Recipe {doc_id} correction failed with OpenAI.")
        if (i > 100): break

ree()


It work ! But it is really slow... <br>
As the Request Per Minute of the API is arround 60, let's make some async request 

In [5]:
import aiohttp
import asyncio


# Measure units
measure_units = [
    "TEASPOON", "TABLESPOON", "CUP", "G", "KG", "L", "ML", "NONE", "EMPTY", "PIECES"
]

example_recipe = {
    "recipeId": "example_id",
    "ingredients": [
        {"quantity": 500.0, "measure": "G", "ingredient": {"vegetarian": False, "vegan": False, "id": "", "name": "Ground beef"}},
        {"quantity": 500.0, "measure": "G", "ingredient": {"vegetarian": False, "vegan": False, "id": "", "name": "Tomato sauce"}},
        {"quantity": 1.0, "measure": "CUP", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Ricotta cheese"}},
        {"quantity": 2.0, "measure": "CUP", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Mozzarella cheese, shredded"}},
        {"quantity": 0.5, "measure": "CUP", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Parmesan cheese, grated"}},
        {"quantity": 1.0, "measure": "NONE", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Egg"}},
        {"quantity": 1.0, "measure": "TEASPOON", "ingredient": {"vegetarian": True, "vegan": True, "id": "", "name": "Salt"}},
        {"quantity": 1.0, "measure": "TEASPOON", "ingredient": {"vegetarian": True, "vegan": True, "id": "", "name": "Black pepper"}},
        {"quantity": 12.0, "measure": "NONE", "ingredient": {"vegetarian": True, "vegan": False, "id": "", "name": "Lasagna noodles"}}
    ],
    "tags": ["Italian", "Pasta", "Dinner"],
    "description": "A classic Italian lasagna with layers of ground beef, tomato sauce, and three cheeses.",
    "imageUrl": "example_url",
    "rating": 4.9,
    "title": "Classic Lasagna",
    "steps": [
        {"description": "Preheat oven to 375°F (190°C).", "stepNumber": 1, "title": "Preheat Oven"},
        {"description": "In a large skillet, cook ground beef over medium heat until browned. Add tomato sauce and simmer for 10 minutes.", "stepNumber": 2, "title": "Cook Beef and Sauce"},
        {"description": "In a mixing bowl, combine ricotta cheese, egg, salt, and pepper.", "stepNumber": 3, "title": "Prepare Ricotta Mixture"},
        {"description": "Cook lasagna noodles according to package instructions. Drain and set aside.", "stepNumber": 4, "title": "Cook Noodles"},
        {"description": "In a 9x13 inch baking dish, spread a layer of meat sauce. Add a layer of noodles, followed by a layer of the ricotta mixture, mozzarella cheese, and Parmesan cheese. Repeat layers, ending with mozzarella and Parmesan on top.", "stepNumber": 5, "title": "Assemble Lasagna"},
        {"description": "Cover with aluminum foil and bake in preheated oven for 25 minutes. Remove foil and bake for an additional 25 minutes until cheese is bubbly and slightly golden.", "stepNumber": 6, "title": "Bake Lasagna"},
        {"description": "Let lasagna cool for 10 minutes before serving.", "stepNumber": 7, "title": "Cool and Serve"}
    ],
    "userid": "12345"
}

# Function to create the prompt
def create_prompt(recipe):
    return f"""
    Correct and fill out the following recipe JSON to ensure it contains all required fields, especially a description, step title and the quantity measure unit which should be one of the following {', '.join(measure_units)}. 
    If the measure is NONE, replace this value by a coherent one in function of the ingredient name.
    If the quantity is 0, replace this value by a coherent one in function of the ingredient name.
    If the rating is 0, based on the popularity of the recipe generate a rating.

    Example of the expected format:
    {json.dumps(example_recipe, indent=2)}

    Recipe to correct:
    {json.dumps(recipe, indent=2)}

    Output only the corrected recipe JSON. Ensure the output is valid JSON.
    """

# Function to make a single request
async def correct_recipe(session, recipe):
    prompt = create_prompt(recipe)
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {openai.api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant that corrects recipe formats."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 4000
    }

    try:
        async with session.post(url, json=data, headers=headers) as response:
            if response.status == 200:
                response_json = await response.json()
                corrected_recipe_str = response_json['choices'][0]['message']['content'].strip()

                # Ensure the response is a valid JSON
                start_index = corrected_recipe_str.find('{')
                end_index = corrected_recipe_str.rfind('}') + 1
                corrected_recipe_str = corrected_recipe_str[start_index:end_index]

                try:
                    corrected_recipe = json.loads(corrected_recipe_str)
                    return corrected_recipe, True
                except json.JSONDecodeError as e:
                    print(f"JSON decode error: {e}")
                    print(f"Received invalid JSON: {corrected_recipe_str}")
                    return recipe, False
            else:
                print(f"API request failed with status code: {response.status}")
                return recipe, False
    except Exception as e:
        print(f"Error during API request: {e}")
        return recipe, False

# Function to process recipes concurrently
async def correct_all_recipes(doc_filter):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for recipe in doc_filter:
            task = asyncio.ensure_future(correct_recipe(session, recipe))
            tasks.append(task)
            await asyncio.sleep(5)  # Rate limiting
        responses = await asyncio.gather(*tasks)
        return responses

# Function to update the Firestore database
def update_recipe_in_firestore(doc_id, corrected_recipe):
    db.collection("recipesFinal").document(doc_id).set(corrected_recipe, merge=True)

# Main function to run the asynchronous tasks
async def main_async(doc_filter):
    recipes = await correct_all_recipes(doc_filter)
    
    # Process and update recipes as needed
    for original_recipe, (corrected_recipe, ai_corrected) in zip(doc_filter, recipes):
        doc_id = original_recipe['recipeId']  # Assuming each recipe has a 'recipeId' field
        if ai_corrected:
            print(f"Old recipe {original_recipe}")
            print(f"Updated recipe {corrected_recipe}")
            update_recipe_in_firestore(doc_id, corrected_recipe)
            print(f"Recipe {doc_id} corrected and updated in Firestore.")
        else:
            print(f"Recipe {doc_id} correction failed with OpenAI.")

# Function to run the main_async function in Jupyter Notebook
def run_in_notebook(doc_filter):
    loop = asyncio.get_event_loop()
    if loop.is_running():
        loop.create_task(main_async(doc_filter))
    else:
        loop.run_until_complete(main_async(doc_filter))


In [6]:
run_in_notebook(doc_filter)

JSON decode error: Expecting property name enclosed in double quotes: line 74 column 6 (char 1527)
Received invalid JSON: {
  "recipeId": "46KxAUF48DEyoTmB1spM",
  "ingredients": [
    {
      "quantity": 500.0,
      "measure": "ML",
      "ingredient": {
        "vegetarian": false,
        "vegan": false,
        "id": "fJhycQdlkd0qYkfrKlFr",
        "name": "Broth"
      }
    },
    {
      "quantity": 1.0,
      "measure": "TEASPOON",
      "ingredient": {
        "vegetarian": false,
        "vegan": false,
        "id": "8oCXQCNCxht1FIL81ldv",
        "name": "Black pepper"
      }
    },
    {
      "quantity": 5.0,
      "measure": "ML",
      "ingredient": {
        "vegetarian": false,
        "vegan": false,
        "id": "JR4S8BRRxqfRMr6AqzcP",
        "name": "Thyme"
      }
    },
    {
      "quantity": 2.0,
      "measure": "TEASPOON",
      "ingredient": {
        "vegetarian": false,
        "vegan": false,
        "id": "gdNqyLlXwMtkyPGzbmyX",
        "name": "Salt

Only two recipes failed to be updated ! <br>
The new collection at `recipesFinal` is now well formated :)